In [ ]:
import os
from PIL import Image
import numpy as np
import pickle

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm.notebook import tqdm

from keras import backend as k
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Model

from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception


from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,AveragePooling2D,Dropout,BatchNormalization,Activation
from keras.activations import relu,softmax
from keras.initializers import he_normal,glorot_normal,random_normal,glorot_uniform
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta,Adam
from keras.utils import np_utils

import warnings
warnings.filterwarnings('ignore')


In [ ]:
!nvidia-smi

Tue Apr 13 20:00:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip

--2021-04-13 20:02:08--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.203.128, 74.125.20.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G  24.2MB/s    in 67s     

2021-04-13 20:03:15 (54.5 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [ ]:
! unzip '/content/nature_12K.zip'

In [ ]:
class Fine_Tune:

  #target_size = (w x h x channel)
  def __init__(self,target_size, num_neurons,num_hlayers,pretrained_model,trainable_layers):
    self.model = Sequential()
    self.target_size = target_size
    self.num_neurons = num_neurons
    self.num_hlayers = num_hlayers
    self.pretrained_model = pretrained_model
    self.trainable_layers = trainable_layers

  def BUILD_MODEL(self):
    if (self.pretrained_model == 'InceptionV3'):
      prt = InceptionV3(weights='imagenet',include_top=False,input_shape = self.target_size)
      trl = 17
      
    elif (self.pretrained_model == 'InceptionResnetV2'):
      prt = InceptionResNetV2(weights='imagenet',include_top=False,input_shape = self.target_size)
      trl = 17

    elif (self.pretrained_model == 'ResNet50'):
      prt = ResNet50(weights='imagenet',include_top=False,input_shape = self.target_size)
      trl = 10

    elif (self.pretrained_model == 'Xception') :
      prt = Xception(weights='imagenet',include_top=False,input_shape = self.target_size)
      trl = 6
    else:
      raise Exception("this model not used!!")
   
    if self.trainable_layers == 'No':
      for layer in prt.layers:
          layer.trainable = False
    else:
      for layer in prt.layers[:-1*trl]:
          layer.trainable = False

    self.model.add(prt)

    #Add dense layers.. 
    self.model.add(Flatten())

    for i in range(self.num_hlayers):
      self.model.add(Dense(units = self.num_neurons))
      self.model.add(Activation('relu'))

    self.model.add(Dense(units=10))
    self.model.add(Activation('softmax'))

    self.model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

    return


  def Fit_Model(self,train_path,epochs,batch_size):
    tr_datagen = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.2,
                                    horizontal_flip=True,
                                    validation_split = 0.2)

    tr_gen = tr_datagen.flow_from_directory(
            directory = train_path,target_size = self.target_size[:2],color_mode = 'rgb',       
            batch_size = batch_size,
            class_mode = 'categorical',
            shuffle = True,
            seed = 10,
            subset = 'training'
            )

    val_gen = tr_datagen.flow_from_directory(
            directory = train_path,target_size = self.target_size[:2],color_mode = 'rgb',
            batch_size = batch_size,
            class_mode = 'categorical',
            shuffle = True,
            seed = 10,
            subset = 'validation'
            ) 
   
    step_size_train = tr_gen.n//tr_gen.batch_size
    step_size_valid = val_gen.n//val_gen.batch_size

    self.model.fit_generator(generator = tr_gen,
                             steps_per_epoch = step_size_train,
                             validation_data = val_gen,
                             validation_steps = step_size_valid,
                             epochs = epochs,
                            callbacks=[WandbCallback(monitor='val_accuracy',save_model=True)],
                             verbose = 1)
    return

  def Predict_Test(self,test_path):

    ts_datagen = ImageDataGenerator(rescale = 1./255)
    test_gen = ts_datagen.flow_from_directory(
        directory = test_path,
        target_size = self.target_size[:2],
        color_mode = 'rgb',
        batch_size = 1,
        class_mode = None,
        shuffle = False,
        seed=10
        )
    
    step_size_test = test_gen.n//test_gen.batch_size
    test_gen.reset()
    print('Getting predictions for test data..')
    pred = self.model.predict_generator(generator = test_gen,
                                  steps = step_size_test,
                                  verbose = 1
                                  )
    ytrue = test_gen.classes ; ypred = np.argmax(pred,axis=1)
    return np.sum(ytrue == ypred)/len(ytrue)

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

     |████████████████████████████████| 2.1MB 18.0MB/s 
     |████████████████████████████████| 163kB 55.0MB/s 
     |████████████████████████████████| 133kB 58.7MB/s 
     |████████████████████████████████| 102kB 14.6MB/s 
     |████████████████████████████████| 71kB 11.7MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

**Question 2 (5 Marks)
You will notice that InceptionV3, InceptionResNetV2, ResNet50, Xception are very huge models as compared to the simple model that you implemented in Part A. Even fine-tuning on a small training data may be very expensive. What is a common trick used to keep the training tractable (you will have to read up a bit on this)? Try different variants of this trick and fine-tune the model using the iNaturalist dataset. For example, '___'ing all layers except the last layer, '___'ing upto k layers and '___'ing the rest. Read up on pre-training and fine-tuning to understand what exactly these terms mean.**

**Write down the different strategies that you tried (simple bullet points would be fine).**

In [ ]:
sweep_config = {
  'name': 'Sweep for Assignment PartB',
  'method': 'grid',
  'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
  'parameters': {
        'num_neurons': {
            'values': [64,128]
        },
        'num_hlayers':{
            'values':[1,2]
        },
        'pretrained_model':{
            'values':['InceptionV3','Xception','InceptionResnetV2','ResNet50']
        },
        'trainable_layers':{
            'values':['Yes','No']
        }

    }
}

sweep_id = wandb.sweep(sweep_config, project='CNN_PartB', entity='manideepladi')

Create sweep with ID: 8umpr1gn
Sweep URL: https://wandb.ai/manideepladi/CNN_PartB/sweeps/8umpr1gn


In [ ]:
import wandb
from wandb.keras import WandbCallback

In [ ]:
def train():
  run = wandb.init()
  config = run.config

  train_path = '/content/inaturalist_12K/train'

  target_size = (256,256,3)

  obj1 = Fine_Tune(target_size=target_size ,
                   pretrained_model = config.pretrained_model,
                   num_neurons = config.num_neurons,
                   num_hlayers = config.num_hlayers,
                   trainable_layers=config.trainable_layers) 
                       
  obj1.BUILD_MODEL()
  obj1.Fit_Model(train_path,epochs=10,batch_size=32)

sweep_id="mbq45fhk"
wandb.agent(sweep_id=sweep_id, function=train )


wandb: Agent Starting Run: f511lkq6 with config:
wandb: 	num_hlayers: 2
wandb: 	num_neurons: 128
wandb: 	pretrained_model: Xception
wandb: 	trainable_layers: Yes
wandb: Currently logged in as: manideepladi (use `wandb login --relogin` to force relogin)


83689472/83683744 [==============================] - 1s 0us/step
Found 8000 images belonging to 10 classes.
Found 1999 images belonging to 10 classes.
Epoch 1/10
250/250 [==============================] - 309s 1s/step - loss: 1.4975 - accuracy: 0.6118 - val_loss: 0.8091 - val_accuracy: 0.7434
Epoch 2/10
250/250 [==============================] - 275s 1s/step - loss: 0.5957 - accuracy: 0.8119 - val_loss: 0.8709 - val_accuracy: 0.7485
Epoch 3/10
250/250 [==============================] - 264s 1s/step - loss: 0.4657 - accuracy: 0.8445 - val_loss: 0.8410 - val_accuracy: 0.7742
Epoch 4/10
250/250 [==============================] - 266s 1s/step - loss: 0.3256 - accuracy: 0.8961 - val_loss: 0.8842 - val_accuracy: 0.7651
Epoch 5/10
250/250 [==============================] - 266s 1s/step - loss: 0.2627 - accuracy: 0.9166 - val_loss: 1.0325 - val_accuracy: 0.7732
Epoch 6/10
250/250 [==============================] - 265s 1s/step - loss: 0.1904 - accuracy: 0.9435 - val_loss: 1.0662 - val_accuracy

epoch,9
loss,0.14158
accuracy,0.9545
val_loss,1.0904
val_accuracy,0.7752
_runtime,2718
_timestamp,1618292711
_step,9
best_val_accuracy,0.7752
best_epoch,9


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▅▄▃▂▂▁▁▁▁
accuracy,▁▄▅▆▇▇████
val_loss,▁▂▂▂▅▅▆▆█▆
val_accuracy,▁▂█▆█▆▃▅██
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: drwkrbnh with config:
wandb: 	num_hlayers: 2
wandb: 	num_neurons: 128
wandb: 	pretrained_model: Xception
wandb: 	trainable_layers: No


...

94773248/94765736 [==============================] - 1s 0us/step
Found 8000 images belonging to 10 classes.
Found 1999 images belonging to 10 classes.
Epoch 1/10
250/250 [==============================] - 265s 1s/step - loss: 4.4492 - accuracy: 0.1183 - val_loss: 2.2798 - val_accuracy: 0.1366
Epoch 2/10
250/250 [==============================] - 258s 1s/step - loss: 2.2569 - accuracy: 0.1659 - val_loss: 2.2485 - val_accuracy: 0.1749
Epoch 3/10
250/250 [==============================] - 260s 1s/step - loss: 2.2122 - accuracy: 0.1845 - val_loss: 2.2273 - val_accuracy: 0.1820
Epoch 4/10
250/250 [==============================] - 262s 1s/step - loss: 2.1980 - accuracy: 0.1940 - val_loss: 2.3013 - val_accuracy: 0.1779
Epoch 5/10
250/250 [==============================] - 260s 1s/step - loss: 2.1634 - accuracy: 0.1935 - val_loss: 2.2315 - val_accuracy: 0.1825
Epoch 6/10
250/250 [==============================] - 262s 1s/step - loss: 2.1503 - accuracy: 0.2066 - val_loss: 2.1372 - val_accuracy

epoch,9
loss,2.13991
accuracy,0.202
val_loss,2.26672
val_accuracy,0.14315
_runtime,2604
_timestamp,1618303476
_step,9
best_val_accuracy,0.22732
best_epoch,5


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▂▂▂▁▁▁▁▁▁
accuracy,▁▄▅▆▆▇▇█▆▇
val_loss,▂▂▂▂▂▁█▃▂▂
val_accuracy,▃▅▅▅▅█▁▄▃▃
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: kpqqu0q1 with config:
wandb: 	num_hlayers: 2
wandb: 	num_neurons: 128
wandb: 	pretrained_model: ResNet50
wandb: 	trainable_layers: No


Found 8000 images belonging to 10 classes.
Found 1999 images belonging to 10 classes.
Epoch 1/10
250/250 [==============================] - 262s 1s/step - loss: 3.9314 - accuracy: 0.1218 - val_loss: 2.4581 - val_accuracy: 0.1431
Epoch 2/10
250/250 [==============================] - 256s 1s/step - loss: 2.2835 - accuracy: 0.1520 - val_loss: 2.3028 - val_accuracy: 0.1003
Epoch 3/10
250/250 [==============================] - 258s 1s/step - loss: 2.3022 - accuracy: 0.1006 - val_loss: 2.3025 - val_accuracy: 0.1003
Epoch 4/10
250/250 [==============================] - 254s 1s/step - loss: 2.3020 - accuracy: 0.1026 - val_loss: 2.2973 - val_accuracy: 0.1069
Epoch 5/10
 36/250 [===>..........................] - ETA: 2:55 - loss: 2.2966 - accuracy: 0.0948

Training with one of the best model from sweep

In [ ]:
train_path = '/content/inaturalist_12K/train'

target_size = (256,256,3)

obj1 = Fine_Tune(target_size=target_size ,
                   pretrained_model = "InceptionResnetV2",
                   num_neurons =128,
                   num_hlayers = 2,
                   trainable_layers="Yes") 
obj1.BUILD_MODEL()
obj1.Fit_Model(train_path,epochs=10,batch_size=32)

Found 8000 images belonging to 10 classes.
Found 1999 images belonging to 10 classes.
Epoch 1/10
250/250 [==============================] - 296s 1s/step - loss: 2.1951 - accuracy: 0.6250 - val_loss: 0.8463 - val_accuracy: 0.7344
Epoch 2/10
250/250 [==============================] - 243s 974ms/step - loss: 0.6010 - accuracy: 0.8123 - val_loss: 1.5475 - val_accuracy: 0.6477
Epoch 3/10
250/250 [==============================] - 237s 947ms/step - loss: 0.4600 - accuracy: 0.8483 - val_loss: 0.9939 - val_accuracy: 0.7278
Epoch 4/10
250/250 [==============================] - 235s 941ms/step - loss: 0.3855 - accuracy: 0.8776 - val_loss: 0.7967 - val_accuracy: 0.7777
Epoch 5/10
250/250 [==============================] - 237s 949ms/step - loss: 0.3172 - accuracy: 0.8951 - val_loss: 0.7527 - val_accuracy: 0.7707
Epoch 6/10
250/250 [==============================] - 236s 945ms/step - loss: 0.2823 - accuracy: 0.9105 - val_loss: 0.9141 - val_accuracy: 0.7772
Epoch 7/10
250/250 [=====================

In [ ]:
#Return the accuracy and predictions on test data ..

test_path = '/content/inaturalist_12K/val'
test_auc = obj1.Predict_Test(test_path)
print(test_auc)

Found 2000 images belonging to 10 classes.
Getting predictions for test data..
2000/2000 [==============================] - 69s 33ms/step
0.777
